In [ ]:
! pip install selenium

  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 15.2 MB/s eta 0:00:00a 0:00:01
Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
Using cached attrs-24.2.0-py3-none-any.whl (63 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0


In [ ]:
# Initialize the WebDriver
driver = webdriver.Chrome()

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from time import sleep

# Setup options
options = Options()
options.add_argument("--headless")  # Temporarily disabling headless mode for debugging
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Initialize WebDriver
driver_service = Service(r"/Users/devanshu29bansalgmail.com/Downloads/chromedriver-mac-arm64/chromedriver")



# Search keywords
search_keywords = ['Face Wash', 'Face Scrub', 'moisturizer', 'sunscreen', 'body lotion', 
                   'Body Wash','face Pack','Dtan Pack' ,'face toner','Face wipes','Face Serum',"Face Creame","Face Gel"]
#search_keywords=['Skincare Products']
platforms = ["Flipkart"]

# Lists to store scraped data
data = []

# Function to scrape Flipkart
def scrape_flipkart():

    for keyword in search_keywords:
        driver = webdriver.Chrome(service=driver_service, options=options)
        driver.get("http://www.flipkart.com")
        sleep(2)
    
    # Close login popup if present
        try:
            close_popup = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'✕')]"))
            )
            close_popup.click()
        except Exception:
            print("Popup not found or not needed")

    
        print(f"Searching for {keyword}...")
        
        # Search for the product
        try:
            search_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.NAME, "q"))
            )
            search_box.clear()
            sleep(5)
            search_box.send_keys(keyword)
            search_box.submit()
            sleep(2)
        except Exception as e:
            print(f"Error finding search box: {e}")
            continue

        for page in range(1, 35):  # Scrape the first 35 pages
            print(f"Scraping page {page} for {keyword}...")

            try:
                # Wait for product elements to load
                products = WebDriverWait(driver, 15).until(
                    EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'slAVV4')]"))
                )
            except Exception as e:
                print(f"Error loading products on page {page}: {e}")
                break

            for product in products:
                try:
                    # Update XPath according to current HTML structure
                    title = product.find_element(By.XPATH, ".//a[@class='wjcEIp']").get_attribute("title")
                    price = product.find_element(By.XPATH, ".//div[contains(@class, 'Nx9bqj')]").text
                    try: 
                        rating = product.find_element(By.XPATH, ".//div[contains(@class, 'XQDdHH')]").text
                    except:
                        rating = "N/A"
                        

                    brand = title.split()[0]  # Extract brand from title (basic logic)
                    product_type = keyword  # Use search keyword as product type
                    packing = product.find_element(By.XPATH, ".//div[contains(@class, 'NqpwHC')]").text

                    # Append the scraped data to the list
                    data.append({
                        "Product": title,
                        "Price": price,
                        "Rating": rating,
                        "Brand": brand,
                        "Product Type": product_type,
                        "Platform": "Flipkart",
                        "Packing": packing
                    })

                except Exception as e:
                    print(f"Error scraping product: {e}")

            # Go to the next page if available
            try:
                next_button=driver.find_elements(By.XPATH, "//span[text()='Next']")
                next_button[0].click()
                sleep(10)
            except Exception:
                print(f"Next button not found on page {page}")
                break

# Scrape Flipkart data
scrape_flipkart()

# Save the data to a DataFrame and export to CSV
df = pd.DataFrame(data)
df.to_csv("skincare_producttttts.csv", index=False)

# Close the driver
driver.quit()


Popup not found or not needed
Searching for Face Wash...
Scraping page 1 for Face Wash...
Scraping page 2 for Face Wash...
Scraping page 3 for Face Wash...
Scraping page 4 for Face Wash...


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(data)
df.to_csv("skincare_products.csv", index=False)